In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
import time
from datetime import date, timedelta
import datetime
from tqdm import tqdm
from interruptingcow import timeout
from interruptingcow import timeout
import re
import string
punctuation_string = string.punctuation
tqdm.pandas()
import gc

Top 100 products from dataset
plus news channels
plus Martin Lewis
What else?

In [2]:
df = pd.read_csv('Datasetjson_small.csv', low_memory=False)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28028 entries, 0 to 28027
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         28028 non-null  float64
 1   vote            4509 non-null   object 
 2   verified        28028 non-null  bool   
 3   reviewTime      28028 non-null  object 
 4   reviewerID      28028 non-null  object 
 5   asin            28028 non-null  object 
 6   reviewerName    28026 non-null  object 
 7   reviewText      28018 non-null  object 
 8   summary         28020 non-null  object 
 9   unixReviewTime  28028 non-null  int64  
 10  style           17471 non-null  object 
 11  image           317 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 2.4+ MB


In [3]:
df_meta= pd.read_csv('meta.csv', low_memory=False)
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14751104 entries, 0 to 14751103
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   brand     object
 1   asin      object
 2   category  object
dtypes: object(3)
memory usage: 337.6+ MB


In [4]:
df['review'] = df['reviewText']+' '+df['summary']
df['review'].head()

0    I bought two of these 8.5 fl oz hand cream, an...
1    Believe me, over the years I have tried many, ...
2                         Great hand lotion Five Stars
3    This is the best for the severely dry skin on ...
4    The best non- oily hand cream ever. It heals o...
Name: review, dtype: object

In [5]:
df.groupby('asin').count()
df[df['asin']=='0209688726']

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review
1001,4.0,3,True,"08 12, 2015",A3ELV1VUCZZGL4,0209688726,AnaG,it's fine. I just would like the stickers to b...,Serve its purpose,1439337600,{'Color:': ' CA'},['https://images-na.ssl-images-amazon.com/imag...,it's fine. I just would like the stickers to b...
1002,2.0,NaN,True,"08 12, 2015",A2FD3FZ2WV6KHQ,0209688726,Obi John Kenobi,took me three returns to get one that didn't w...,and it's still not super stable but whatever,1439337600,{'Color:': ' T'},NaN,took me three returns to get one that didn't w...
1003,2.0,12,True,"08 11, 2015",A39NRKABAVSVSE,0209688726,MuddyRiverGirl,While the product is fine the description and ...,Beware! Description is wrong.,1439251200,{'Color:': ' AC'},['https://images-na.ssl-images-amazon.com/imag...,While the product is fine the description and ...
1004,5.0,NaN,True,"07 29, 2015",AQSBL5KODHSGW,0209688726,Nate S,It's beautiful and blends right in with my woo...,"Awesome desk stand, simple and elegant",1438128000,{'Color:': ' T'},NaN,It's beautiful and blends right in with my woo...
1005,5.0,NaN,False,"07 11, 2015",AU45OQPK2J5JF,0209688726,Kort,I love this stand! I had been looking around f...,I love this stand,1436572800,{'Color:': ' T'},NaN,I love this stand! I had been looking around f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,2.0,NaN,True,"09 30, 2016",A266AWXVTRH5YA,0209688726,Amazon Customer,The magnet is incredibly poor. But cute and ti...,Meh,1475193600,{'Color:': ' CA'},NaN,The magnet is incredibly poor. But cute and ti...
1998,4.0,NaN,False,"09 30, 2016",A1YYEDAWMFHWW0,0209688726,NicoleR,This diffuser necklace is ok. It has more of a...,The smell never really permeates though like i...,1475193600,{'Color:': ' M'},NaN,This diffuser necklace is ok. It has more of a...
1999,5.0,124,True,"09 29, 2016",AM4260SVQHMX4,0209688726,C3,"<div id=""video-block-R3NWBOL144TPGW"" class=""a-...",Good handheld vacuum to have in every car,1475107200,{'Color:': ' Black'},['https://images-na.ssl-images-amazon.com/imag...,"<div id=""video-block-R3NWBOL144TPGW"" class=""a-..."
2000,5.0,NaN,True,"09 29, 2016",AAWZHPPMQN0GC,0209688726,Dannie Hedgpeth,Got the job done,Five Stars,1475107200,{'Color:': ' Black'},NaN,Got the job done Five Stars


In [6]:
df_meta = df_meta.drop_duplicates(subset=['asin', 'brand', 'category'])
df_meta

,brand,asin,category
0,BubuBibi,0000032042,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
1,BubuBibi,0000032069,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
2,Unknown,0000031860,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
3,Tutu,0000031852,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
4,Bububibi,0000031895,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
...,...,...,...
14751099,NaN,B01GMFJ8JU,"['Gift Cards', 'Gift Cards']"
14751100,QuickGifts,B01GOQIKRI,"['Gift Cards', 'Gift Cards']"
14751101,QuickGifts,B01GOQHGL4,"['Gift Cards', 'Gift Cards']"
14751102,Go Play Golf by Fairway Rewards,B01GP1W4LA,"['Gift Cards', 'Gift Cards']"


In [7]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14064249 entries, 0 to 14751103
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   brand     object
 1   asin      object
 2   category  object
dtypes: object(3)
memory usage: 429.2+ MB


In [8]:
df_merge = df.merge(df_meta, on='asin')

In [9]:
df_merge

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,brand,category
0,2.0,3,True,"06 15, 2010",A1Q6MUU0B2ZDQG,B00004U9V2,D. Poston,"I bought two of these 8.5 fl oz hand cream, an...",dispensers don't work,1276560000,NaN,NaN,"I bought two of these 8.5 fl oz hand cream, an...",NaN,[]
1,5.0,14,True,"01 7, 2010",A3HO2SQDCZIE9S,B00004U9V2,chandra,"Believe me, over the years I have tried many, ...",Best hand cream ever.,1262822400,NaN,NaN,"Believe me, over the years I have tried many, ...",NaN,[]
2,5.0,NaN,True,"04 18, 2018",A2EM03F99X3RJZ,B00004U9V2,Maureen G,Great hand lotion,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN,Great hand lotion Five Stars,NaN,[]
3,5.0,NaN,True,"04 18, 2018",A3Z74TDRGD0HU,B00004U9V2,Terry K,This is the best for the severely dry skin on ...,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN,This is the best for the severely dry skin on ...,NaN,[]
4,5.0,NaN,True,"04 17, 2018",A2UXFNW9RTL4VM,B00004U9V2,Patricia Wood,The best non- oily hand cream ever. It heals o...,I always have a backup ready.,1523923200,{'Size:': ' 3.5 oz.'},NaN,The best non- oily hand cream ever. It heals o...,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23707,3.0,NaN,True,"03 15, 2013",A25PFYRQYOVPLH,B000FBFMVG,Scott Spell,Nice to see Chewie in a lead role. I recommend...,A fun change of pace.,1363305600,NaN,NaN,Nice to see Chewie in a lead role. I recommend...,Troy Denning,"['Kindle Store', 'Kindle eBooks', 'Literature ..."
23708,3.0,NaN,True,"01 29, 2013",A1YJE0NK4P0KZJ,B000FBFMVG,blaze,It was great to no that Chewbacca was a family...,Fun read,1359417600,NaN,NaN,It was great to no that Chewbacca was a family...,Troy Denning,"['Kindle Store', 'Kindle eBooks', 'Literature ..."
23709,3.0,NaN,True,"12 20, 2012",A39OV2VJUXKDDI,B000FBFMVG,Keith Meehan,A nice family oriented short featuring Chewbac...,Not bad needs more than just Wookies,1355961600,NaN,NaN,A nice family oriented short featuring Chewbac...,Troy Denning,"['Kindle Store', 'Kindle eBooks', 'Literature ..."
23710,4.0,NaN,False,"07 28, 2012",A2QW0IMOX4EULQ,B000FBFMVG,Amazon Customer,Chewbacca fans should rejoice. Not only does t...,A Forest Apart,1343433600,NaN,NaN,Chewbacca fans should rejoice. Not only does t...,Troy Denning,"['Kindle Store', 'Kindle eBooks', 'Literature ..."


In [10]:
df_merge[df_merge.brand.notnull()].groupby('brand').count().sort_values(by='reviewText', ascending=False)

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,category
brand,,,,,,,,,,,,,,
Visit Amazon's Zondervan Page,828,121,828,828,828,828,828,828,827,828,828,13,827,828
Microsoft,772,136,772,772,772,772,772,772,772,772,0,4,772,772
Alfred Music Publishing,767,52,767,767,767,767,766,767,767,767,15,8,767,767
Amazon,696,140,696,696,696,696,696,695,696,696,657,1,695,696
Trans-Siberian Orchestra,649,109,649,649,649,649,649,649,649,649,649,0,649,649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Monroe Products,1,1,1,1,1,1,1,1,1,1,1,0,1,1
NRS,1,1,1,1,1,1,1,1,1,1,1,0,1,1
Del Monte,1,0,1,1,1,1,1,1,1,1,0,0,1,1


In [11]:
df_merge.dropna().reset_index(drop=True)

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,brand,category
0,3.0,50,True,"03 23, 2016",A38JQFWHLEQO4T,0310432065,AE,Pros:\nWell made.\nThe handle is not on the sp...,"The fabric is like velcro, collecting white fuzz",1458691200,{'Format:': ' Diary'},['https://images-na.ssl-images-amazon.com/imag...,Pros:\nWell made.\nThe handle is not on the sp...,Visit Amazon's Zondervan Page,"['Office Products', 'Office &amp; School Suppl..."
1,5.0,11,True,"06 23, 2016",A3L53VVTN9BXJY,0310520347,Kindle Customer,"An accurate, detailed description and video ar...",Bought this as a journal cover for travel,1466640000,{'Format:': ' Flexibound'},['https://images-na.ssl-images-amazon.com/imag...,"An accurate, detailed description and video ar...",Visit Amazon's Zondervan Page,"['Office Products', 'Office &amp; School Suppl..."
2,5.0,8,True,"08 3, 2016",A3T9QQWESK2RG3,0310802636,Stacey Pollock,Perfect for our child's bible!! They were thri...,So excited for church!!!,1470182400,{'Format:': ' Misc. Supplies'},['https://images-na.ssl-images-amazon.com/imag...,Perfect for our child's bible!! They were thri...,Visit Amazon's Zondervan Page,"['Office Products', 'Office &amp; School Suppl..."
3,5.0,14,True,"03 14, 2017",A2KA4W341XFVSX,0310806607,Danielle,Great quality and fits the Boy Scout book nice...,Great for Boy Scout book,1489449600,{'Format:': ' Misc. Supplies'},['https://images-na.ssl-images-amazon.com/imag...,Great quality and fits the Boy Scout book nice...,Visit Amazon's Zondervan Page,"['Office Products', 'Office & School Supplies'..."
4,5.0,14,True,"02 11, 2018",AVV4P9HCY9XHY,0310818621,judy,"<div id=""video-block-R1WJY4FGKHFPTY"" class=""a-...",Its roomy!!,1518307200,{'Format:': ' Misc. Supplies'},['https://images-na.ssl-images-amazon.com/imag...,"<div id=""video-block-R1WJY4FGKHFPTY"" class=""a-...",Visit Amazon's Zondervan Page,"['Office Products', 'Office &amp; School Suppl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,5.0,10,True,"09 16, 2016",A1DLG8VDEKCEOD,193362275X,Kerri,This little book light is exactly what I was l...,Perfect little book light!,1473984000,{'Format:': ' Kitchen'},['https://images-na.ssl-images-amazon.com/imag...,This little book light is exactly what I was l...,Mighty Bright,"['Tools & Home Improvement', 'Lighting & Ceili..."
75,5.0,2,True,"09 16, 2016",A2ZRPFRHS7M906,193362275X,Amazon Customer,very bright! I use it to power my solar powere...,I use it to power my solar powered desk access...,1473984000,{'Format:': ' Kitchen'},['https://images-na.ssl-images-amazon.com/imag...,very bright! I use it to power my solar powere...,Mighty Bright,"['Tools & Home Improvement', 'Lighting & Ceili..."
76,3.0,8,True,"06 14, 2016",ACCRR97N0CJMA,193362275X,Mariano Ospina,Ok well this is certainly mighty and bright - ...,Camera doesn't capture how white the light it,1465862400,{'Format:': ' Kitchen'},['https://images-na.ssl-images-amazon.com/imag...,Ok well this is certainly mighty and bright - ...,Mighty Bright,"['Tools & Home Improvement', 'Lighting & Ceili..."
77,1.0,6,True,"02 9, 2016",A1L6P4SOP1DFE0,193362275X,Alina Manoochehri,I'm posting this review before even trying thi...,"Received the ""new"" product in an open box!",1454976000,{'Format:': ' Tools & Home Improvement'},['https://images-na.ssl-images-amazon.com/imag...,I'm posting this review before even trying thi...,Mighty Bright,"['Tools & Home Improvement', 'Lighting & Ceili..."


In [12]:
df_merge[df_merge['brand'].notnull()]

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,brand,category
1001,3.0,6,False,"11 3, 2006",A2WJLOXXIB7NF3,0140503528,Brandi D&#039;Angelo,"I thought the book was entertaining and cute, ...",Entertaining-but maybe not for young toddlers,1162512000,{'Format:': ' Hardcover'},NaN,"I thought the book was entertaining and cute, ...",Ingram Book & Distributor,"['Office Products', 'Office & School Supplies'..."
1002,5.0,NaN,False,"05 9, 2006",A1RKICUK0GG6VF,0140503528,Karen Mallory,This adorable story is an all time favorite fa...,adorable!,1147132800,{'Format:': ' Hardcover'},NaN,This adorable story is an all time favorite fa...,Ingram Book & Distributor,"['Office Products', 'Office & School Supplies'..."
1003,5.0,NaN,False,"03 11, 2006",A1QA5E50M398VW,0140503528,Jennifer Edwards,Lisa's bear Corduroy gets lost in the laundrom...,A sequel to the classic,1142035200,{'Format:': ' Paperback'},NaN,Lisa's bear Corduroy gets lost in the laundrom...,Ingram Book & Distributor,"['Office Products', 'Office & School Supplies'..."
1004,5.0,12,False,"01 24, 2001",A3N0HBW8IP8CZQ,0140503528,Melissa P. Cooper,In this installment of Corduroy's adventures w...,More adventures with Lisa's very own bear. . .,980294400,{'Format:': ' Paperback'},NaN,In this installment of Corduroy's adventures w...,Ingram Book & Distributor,"['Office Products', 'Office & School Supplies'..."
1005,5.0,65,False,"07 30, 2000",A1K1JW1C5CUSUZ,0140503528,Donald Mitchell,Researchers constantly find that reading to ch...,Read to Your Child to Create Bonding and Intel...,964915200,{'Format:': ' Paperback'},NaN,Researchers constantly find that reading to ch...,Ingram Book & Distributor,"['Office Products', 'Office & School Supplies'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23707,3.0,NaN,True,"03 15, 2013",A25PFYRQYOVPLH,B000FBFMVG,Scott Spell,Nice to see Chewie in a lead role. I recommend...,A fun change of pace.,1363305600,NaN,NaN,Nice to see Chewie in a lead role. I recommend...,Troy Denning,"['Kindle Store', 'Kindle eBooks', 'Literature ..."
23708,3.0,NaN,True,"01 29, 2013",A1YJE0NK4P0KZJ,B000FBFMVG,blaze,It was great to no that Chewbacca was a family...,Fun read,1359417600,NaN,NaN,It was great to no that Chewbacca was a family...,Troy Denning,"['Kindle Store', 'Kindle eBooks', 'Literature ..."
23709,3.0,NaN,True,"12 20, 2012",A39OV2VJUXKDDI,B000FBFMVG,Keith Meehan,A nice family oriented short featuring Chewbac...,Not bad needs more than just Wookies,1355961600,NaN,NaN,A nice family oriented short featuring Chewbac...,Troy Denning,"['Kindle Store', 'Kindle eBooks', 'Literature ..."
23710,4.0,NaN,False,"07 28, 2012",A2QW0IMOX4EULQ,B000FBFMVG,Amazon Customer,Chewbacca fans should rejoice. Not only does t...,A Forest Apart,1343433600,NaN,NaN,Chewbacca fans should rejoice. Not only does t...,Troy Denning,"['Kindle Store', 'Kindle eBooks', 'Literature ..."


In [13]:
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuation_string)
    return text.translate(translator)

#df_merge = df_merge.dropna().reset_index(drop=True)
df_merge = df_merge[df_merge['brand'].notnull()]
print('cleaning punctuations')
clean_punct = df_merge['brand'].progress_apply(lambda x: cleaning_punctuations(x))
df_merge = df_merge.assign(brand=clean_punct)

cleaning punctuations


100%|██████████| 21656/21656 [00:00<00:00, 64938.43it/s]


In [14]:
remove_visit = df_merge['brand'].progress_apply(lambda x: re.sub('Visit Amazons ','', x))
df_merge= df_merge.assign(brand=remove_visit)

100%|██████████| 21656/21656 [00:00<00:00, 126060.80it/s]


In [15]:
remove_amazonpage = df_merge['brand'].progress_apply(lambda x: re.sub(' Page','', x))

100%|██████████| 21656/21656 [00:00<00:00, 317797.22it/s]


In [16]:
df_merge= df_merge.assign(brand=remove_amazonpage)

In [17]:
df_merge.groupby('brand').count()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,category
brand,,,,,,,,,,,,,,
13Rainbow,9,2,9,9,9,9,9,9,9,9,0,0,9,9
3x5fFlags,118,5,118,118,118,118,118,118,118,118,0,0,118,118
ARTnews LLC,8,5,8,8,8,8,8,8,8,8,0,0,8,8
Aaron Crabb,10,6,10,10,10,10,10,10,10,10,10,0,10,10
Accessory,21,1,21,21,21,21,21,21,21,21,21,1,21,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
by\n \n Star Publications Languages of the World Publications,12,5,12,12,12,12,12,12,12,12,12,0,12,12
by\n \n Storm City Entertainment,13,7,13,13,13,13,13,13,13,13,0,0,13,13
by\n \n Unknown,72,15,72,72,72,72,72,72,72,72,0,0,72,72


In [18]:
text = 'Visit Amazons Zondervan Page'
re.search('Visit Amazons [a-zA-Z]+ Page', text)

<re.Match object; span=(0, 28), match='Visit Amazons Zondervan Page'>

In [19]:
remove_by = df_merge['brand'].progress_apply(lambda x: re.sub('by\n    \n    ','' ,x))

100%|██████████| 21656/21656 [00:00<00:00, 302124.26it/s]


In [20]:
df_merge= df_merge.assign(brand=remove_by)
df_merge.loc[18624]

overall                                                         4.0
vote                                                              2
verified                                                       True
reviewTime                                               01 5, 2011
reviewerID                                           A1YBGCEU62FPPN
asin                                                     7293000936
reviewerName                                                    PAL
reviewText        After few months of use cable flicker and vide...
summary           After few months of use cable flicker and vide...
unixReviewTime                                           1294185600
style                                                           NaN
image                                                           NaN
review            After few months of use cable flicker and vide...
brand                                                       Unknown
category          ['Video Games', 'Wii', 'Access

In [21]:
df_products = df_merge.groupby('brand').count().sort_values('overall', ascending=False)

In [22]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 257 entries, Zondervan to James Hime
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   overall         257 non-null    int64
 1   vote            257 non-null    int64
 2   verified        257 non-null    int64
 3   reviewTime      257 non-null    int64
 4   reviewerID      257 non-null    int64
 5   asin            257 non-null    int64
 6   reviewerName    257 non-null    int64
 7   reviewText      257 non-null    int64
 8   summary         257 non-null    int64
 9   unixReviewTime  257 non-null    int64
 10  style           257 non-null    int64
 11  image           257 non-null    int64
 12  review          257 non-null    int64
 13  category        257 non-null    int64
dtypes: int64(14)
memory usage: 30.1+ KB


In [23]:
df_products = df_products.rename_axis('brand').reset_index()


In [24]:
top_100 = df_products['brand'].head(100)
top_100_list = top_100.to_list()

top_100_list.remove('Unknown')
top_100_list.remove('nobrand')

In [25]:
top_100_list

['Zondervan',
 'Microsoft',
 'Alfred Music Publishing',
 'Amazon',
 'TransSiberian Orchestra',
 'Holmes',
 'Mighty Bright',
 'Yat Ming',
 'Frank W Abagnale',
 'DRREXY',
 'Magnetic Poetry',
 'Chicken of the Sea',
 'Barnes amp Noble',
 'ELEGANI',
 'Black Mountain',
 'Progresso',
 'Mae Ploy',
 'Gorilla',
 'Logitech',
 'Trusted Media Brands Inc',
 'Conde Nast Publications',
 'SioGreen',
 'DIAMOND SELECT TOYS',
 'PORTERCABLE',
 'Dundas Jafine',
 'TopHeadwear',
 'Jeannette Winters',
 'Cesar Millan',
 'Garmin',
 'Wizards of the Coast',
 'Wiley Publishing',
 'Generic',
 'Klutz',
 'Future US Inc',
 'Therapy Game HQ',
 'Traditional Medicinals',
 'Wilton',
 'Casting Crowns',
 'Anita Shreve',
 'Lipton',
 'Daniel Kruse',
 'Chris Rice',
 'Craft  Empura',
 'Organo Gold',
 'John Michael Talbot',
 'Susan B Anderson',
 '3x5fFlags',
 'Gilmour',
 'Dover Publications',
 'Ben Kingsley',
 'Ingram Book  Distributor',
 'Brak Software',
 'URC',
 'Ethan M Rasiel',
 'Bully Tools',
 'Dirt 3',
 'Jim Wilson',
 'Art 

In [26]:
df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang'])
start_date = date(2015, 12, 31)
end_date = date(2022, 8, 2)

df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    f'{top_100[0]}  since:{start_date} until:{end_date} within: 10km' ).get_items(), 100))
            #[['date', 'content', 'user','hashtags', 'coordinates', 'lang']]

In [27]:
df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    f'{top_100[1]} geocode:53.81653,-3.03033,320km since:2022-07-01 until:2022-07-31').get_items(), 100000))

In [28]:
df_collect['just_date'] = df_collect['date'].dt.date

In [29]:
df_collect.groupby('just_date').count()

,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
just_date,,,,,,,,,,,,,,,,,,,,,
2022-07-01,5,5,5,5,5,5,5,5,5,5,...,1,0,0,4,4,5,5,5,1,0
2022-07-02,5,5,5,5,5,5,5,5,5,5,...,0,0,1,4,4,4,5,5,2,0
2022-07-03,5,5,5,5,5,5,5,5,5,5,...,1,0,2,3,3,4,5,5,1,0
2022-07-04,27,27,27,27,27,27,27,27,27,27,...,6,0,0,23,23,24,27,27,14,0
2022-07-05,26,26,26,26,26,26,26,26,26,26,...,1,0,1,23,23,23,26,26,13,0
2022-07-06,7,7,7,7,7,7,7,7,7,7,...,2,0,1,2,2,5,7,7,4,0
2022-07-07,5,5,5,5,5,5,5,5,5,5,...,2,0,1,3,3,4,5,5,2,0
2022-07-08,109,109,109,109,109,109,109,109,109,109,...,4,0,3,105,105,105,109,109,72,0
2022-07-09,19,19,19,19,19,19,19,19,19,19,...,0,0,1,18,18,19,19,19,16,0


In [31]:
#https://stackoverflow.com/questions/11865685/handling-a-timeout-error-in-python-sockets
import errno
import os
import signal
import functools

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wrapper

    return decorator

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [ ]:
@timeout(60)
def scrape_product(start, end, product):
    df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])
    df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
        f'{product} geocode:53.81653,-3.03033,320km since:{start} until:{end}').get_items(), 100000))
    df_collect['product'] = product
    df_tweets = pd.concat([df_tweets, df_collect])#, df_BBC, df_Amazon
    return df_tweets

start_date = date(2016, 2, 4)
end_date = date(2022, 8, 2)
dates = pd.date_range(start = start_date, end = end_date,freq='D').strftime('%Y-%m-%d').tolist()

count=1
df_collected = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])
df_new = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])
for single_date in tqdm(daterange(start_date, end_date)):
    next_date = single_date + datetime.timedelta(days=1)
    df_new = df_new.drop(df_new.index,inplace=True)
    df_collected = df_collected.drop(df_collected.index,inplace=True)
    for product in tqdm(top_100_list):
        try:
            #print(f'doing, single {single_date} next {next_date} product {product}')
            df_new = scrape_product(single_date, next_date, product)
        except TimeoutError:
            print(f'timeout, single {single_date} next {next_date} product {product}')
            continue
        df_collected = pd.concat([df_collected, df_new])
    df_collected.to_csv(f'tweets/tweets{single_date}.csv', index=False)




0it [00:00, ?it/s]
100%|██████████| 98/98 [02:19<00:00,  1.42s/it]
1it [02:19, 139.12s/it]
100%|██████████| 98/98 [02:09<00:00,  1.32s/it]
2it [04:28, 133.67s/it]
100%|██████████| 98/98 [02:05<00:00,  1.28s/it]
3it [06:34, 129.76s/it]
 16%|█▋        | 16/98 [30:35<3:01:51, 133.07s/it]